In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import GPT2Model, GPT2Config, PreTrainedModel
from datasets import load_dataset, Dataset, DatasetDict

from torch.utils.data import DataLoader
from transformers import Trainer, TrainingArguments

import math

In [2]:
class GPT2Trader(PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        
        # use levine 2020 layer numbers
        n_layer = round((math.log(config.n_embd) - 5.039) / 5.55e-2)
        n_layer = max(1, n_layer)
        print(f'Using {n_layer} layers')
        config.n_layer = n_layer
        
        config.initializer_range = 1 / math.sqrt(config.n_embd)
        
        self.embed = nn.Linear(5, config.n_embd, bias = False)
        self.norm = nn.LayerNorm(config.n_embd)
        self.gpt = GPT2Model(config)
        self.copy = nn.Linear(config.n_embd, config.n_embd, bias = False)
        self.regress = nn.Linear(config.n_embd, 3, bias = False)
        self.classify = nn.Linear(config.n_embd, 2, bias = False)
        self.trade = nn.Linear(config.n_embd, 2, bias = False)
        
        self.MSELoss = nn.MSELoss()
        self.CrossEntropyLoss = nn.CrossEntropyLoss()
        
        self.trade_sign = nn.Parameter(torch.Tensor([1, -1]), requires_grad = False)
        
    def forward(self, ohlcv, future):
        embed = self.norm(self.embed(ohlcv))
        hidden = self.gpt(inputs_embeds = embed).last_hidden_state
        
        copy = self.copy(hidden).reshape(-1, hidden.shape[-1])
        regression_pred = self.regress(hidden).reshape(-1, 3)
        classification = self.classify(hidden).reshape(-1, 2)
        soft_trade = self.trade(hidden).reshape(-1, 2)
        
        # information
        original_close = ohlcv.select(index = 3, dim = -1).unsqueeze(-1)
        future_close = future.select(index = 2, dim = -1).unsqueeze(-1)
        delta = (original_close - future_close).reshape(-1)
        class_sign = (delta >= 0).long()
        
        # sharpe information
        soft_trade = F.softmax(soft_trade, dim = -1)
        soft_trade = soft_trade / (soft_trade.sum(dim = -1).unsqueeze(1) + 1e-5)
        soft_trade = soft_trade * self.trade_sign
        soft_trade = soft_trade.sum(dim = -1).reshape(-1)
        soft_trade = soft_trade * delta
        
        copy_loss = self.MSELoss(copy, embed.reshape(-1, hidden.shape[-1]).detach())
        mse_loss = self.MSELoss(regression_pred, future.reshape(-1, 3)) / 1e-3
        ce_loss = self.CrossEntropyLoss(classification, class_sign)
        sharpe = soft_trade.mean() * (delta.shape[0]**.5) / (soft_trade.std() + 1e-10)
        sharpe_loss = F.elu(-sharpe) + 1
        
        loss = sharpe_loss / 3 + ce_loss / 3 + mse_loss / 3 + copy_loss
        
        print(f"sharpe {sharpe.item():.2}, ce_loss {ce_loss.item():.4}, mse_loss {mse_loss.item():.4}, copy_loss {copy_loss.item():.4}")
        print(f"preds {regression_pred[0].detach(), future.reshape(-1, 3)[0].detach()}")
        
        return {"loss": loss}

In [3]:
class TraderTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         labels = inputs.get("labels")
#         # forward pass
#         outputs = model(**inputs)
#         logits = outputs.get("logits")
#         # compute custom loss (suppose one has 3 labels with different weights)
#         loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 3.0]))
#         loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
#         return (loss, outputs) if return_outputs else loss

    def get_train_dataloader(self) -> DataLoader:
        """
        Returns the training :class:`~torch.utils.data.DataLoader`.

        Will use no sampler if :obj:`self.train_dataset` does not implement :obj:`__len__`, a random sampler (adapted
        to distributed training if necessary) otherwise.

        Subclass and override this method if you want to inject some custom behavior.
        """
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")
        train_sampler = self._get_train_sampler()

        return DataLoader(
            self.train_dataset,
            batch_size=self.args.train_batch_size,
#             shuffle=False, # IMPORTANT TO STOP OVERFITTING
            sampler=train_sampler,
            collate_fn=self.data_collator,
            drop_last=self.args.dataloader_drop_last,
            num_workers=self.args.dataloader_num_workers,
        )

In [4]:
eurusd = Dataset.from_parquet('data/EURUSD_day.pq', )
eurusd = DatasetDict({
    'train': Dataset.from_dict(eurusd[:3000]),
    'validation': Dataset.from_dict(eurusd[3000:3200]),
    'test': Dataset.from_dict(eurusd[3200:])
})

Using custom data configuration default-a5e648de2e586aaa
Found cached dataset parquet (C:/Users/micha/.cache/huggingface/datasets/parquet/default-a5e648de2e586aaa/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [5]:
def step(self, closure = None):
    """
    Performs a single optimization step.
    Arguments:
        closure (`Callable`, *optional*): A closure that reevaluates the model and returns the loss.
    """
    loss = None
    if closure is not None:
        loss = closure()

    for group in self.param_groups:
        for p in group["params"]:
            if p.grad is None:
                continue
#             grad = -p.grad.data # SEE NEGATIVE SIGN FOR MAXIMIZATION
            grad = p.grad.data
            if grad.is_sparse:
                raise RuntimeError("Adam does not support sparse gradients, please consider SparseAdam instead")

            state = self.state[p]

            # State initialization
            if len(state) == 0:
                state["step"] = 0
                # Exponential moving average of gradient values
                state["exp_avg"] = torch.zeros_like(p.data)
                # Exponential moving average of squared gradient values
                state["exp_avg_sq"] = torch.zeros_like(p.data)

            exp_avg, exp_avg_sq = state["exp_avg"], state["exp_avg_sq"]
            beta1, beta2 = group["betas"]

            state["step"] += 1

            # Decay the first and second moment running average coefficient
            # In-place operations to update the averages at the same time
            exp_avg.mul_(beta1).add_(grad, alpha=(1.0 - beta1))
            exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1.0 - beta2)
            denom = exp_avg_sq.sqrt().add_(group["eps"])

            step_size = group["lr"]
            if group["correct_bias"]:  # No bias correction for Bert
                bias_correction1 = 1.0 - beta1 ** state["step"]
                bias_correction2 = 1.0 - beta2 ** state["step"]
                step_size = step_size * math.sqrt(bias_correction2) / bias_correction1

            p.data.addcdiv_(exp_avg, denom, value=-step_size)

            # Just adding the square of the weights to the loss function is *not*
            # the correct way of using L2 regularization/weight decay with Adam,
            # since that will interact with the m and v parameters in strange ways.
            #
            # Instead we want to decay the weights in a manner that doesn't interact
            # with the m/v parameters. This is equivalent to adding the square
            # of the weights to the loss with plain (non-momentum) SGD.
            # Add weight decay at the end (fixed version)
            if group["weight_decay"] > 0.0:
                p.data.add_(p.data, alpha=(-group["lr"] * group["weight_decay"]))

    return loss

from transformers import AdamW
AdamW.step = step

In [6]:
training_args = TrainingArguments(
    output_dir = "./results",
    logging_strategy = "steps",
    evaluation_strategy = "steps",
    logging_steps = 50,
    eval_steps = 50,
    report_to = "none",
    learning_rate = 5e-5,
    lr_scheduler_type = "cosine",
    warmup_ratio = .05,
    num_train_epochs = 1,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    max_grad_norm = 1,
)

In [7]:
config = GPT2Config(
    n_embd = 384, n_head = 6, vocab_size = 0, n_positions = 2000,
#     resid_pdrop = .1, embd_pdrop = .1, attn_pdrop = .1,
    resid_pdrop = 0, embd_pdrop = 0, attn_pdrop = 0,
    scale_attn_by_inverse_layer_idx = True, use_cache = False
)

In [8]:
model = GPT2Trader(config).cuda()
trainer = TraderTrainer(
    model = model,
    args = training_args,
    train_dataset = eurusd['train'],
    eval_dataset = eurusd['validation']
)

Using 16 layers


In [9]:
trainer.train()

C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3000
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 1500


sharpe 2.3e+01, ce_loss 0.8422, mse_loss 580.3, copy_loss 1.293
preds (tensor([1.0617, 0.4326, 0.2934], device='cuda:0'), tensor([1.1062, 1.1057, 1.1061], device='cuda:0'))


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss
50,42.170000,3.153074
100,17.207000,10.890100
150,13.615800,2.110154
200,11.253200,2.981830
250,13.134100,5.295961
300,9.015500,2.010702
350,8.925500,5.800998
400,7.460700,5.096503
450,7.954100,4.426535
500,7.640100,1.911132


sharpe -2.7, ce_loss 0.7619, mse_loss 670.2, copy_loss 1.296
preds (tensor([1.2061, 0.5535, 0.2695], device='cuda:0'), tensor([1.1707, 1.1704, 1.1707], device='cuda:0'))
sharpe -1.2e+01, ce_loss 0.7115, mse_loss 392.9, copy_loss 1.292
preds (tensor([1.2024, 0.5961, 0.3222], device='cuda:0'), tensor([1.0869, 1.0864, 1.0865], device='cuda:0'))
sharpe -3.6, ce_loss 0.7681, mse_loss 451.3, copy_loss 1.29
preds (tensor([0.8862, 0.4900, 0.4494], device='cuda:0'), tensor([1.3611, 1.3599, 1.3600], device='cuda:0'))
sharpe 5.1, ce_loss 0.7317, mse_loss 334.9, copy_loss 1.295
preds (tensor([ 0.6291,  1.1317, -0.1130], device='cuda:0'), tensor([1.2714, 1.2693, 1.2693], device='cuda:0'))
sharpe 7.2, ce_loss 0.8035, mse_loss 72.03, copy_loss 1.28
preds (tensor([0.9786, 0.9038, 0.8101], device='cuda:0'), tensor([1.1840, 1.1833, 1.1837], device='cuda:0'))
sharpe 9.7, ce_loss 0.7994, mse_loss 80.12, copy_loss 1.289
preds (tensor([1.0665, 1.1797, 1.0732], device='cuda:0'), tensor([1.4394, 1.4366, 1.438

...

sharpe -9.8, ce_loss 0.6936, mse_loss 0.1496, copy_loss 0.008007
preds (tensor([1.1709, 1.1757, 1.1795], device='cuda:0'), tensor([1.1782, 1.1778, 1.1779], device='cuda:0'))
sharpe -1.4, ce_loss 0.6931, mse_loss 0.1793, copy_loss 0.008407
preds (tensor([1.1718, 1.1766, 1.1803], device='cuda:0'), tensor([1.1867, 1.1859, 1.1862], device='cuda:0'))
sharpe -2.7, ce_loss 0.6938, mse_loss 0.4638, copy_loss 0.01106
preds (tensor([1.1761, 1.1806, 1.1839], device='cuda:0'), tensor([1.1876, 1.1872, 1.1874], device='cuda:0'))
sharpe -7.5, ce_loss 0.6934, mse_loss 0.4057, copy_loss 0.01227
preds (tensor([1.1789, 1.1832, 1.1862], device='cuda:0'), tensor([1.1905, 1.1896, 1.1905], device='cuda:0'))
sharpe -1.1e+01, ce_loss 0.6938, mse_loss 0.1412, copy_loss 0.005756
preds (tensor([1.1759, 1.1804, 1.1837], device='cuda:0'), tensor([1.1973, 1.1966, 1.1970], device='cuda:0'))
sharpe -7.1, ce_loss 0.6935, mse_loss 0.5288, copy_loss 0.005307
preds (tensor([1.1738, 1.1785, 1.1820], device='cuda:0'), tenso

***** Running Evaluation *****
  Num examples = 200
  Batch size = 2


sharpe -9.8, ce_loss 0.6936, mse_loss 0.1461, copy_loss 0.008
preds (tensor([1.1715, 1.1758, 1.1789], device='cuda:0'), tensor([1.1782, 1.1778, 1.1779], device='cuda:0'))
sharpe -1.4, ce_loss 0.6931, mse_loss 0.1759, copy_loss 0.0084
preds (tensor([1.1725, 1.1767, 1.1797], device='cuda:0'), tensor([1.1867, 1.1859, 1.1862], device='cuda:0'))
sharpe -2.7, ce_loss 0.6938, mse_loss 0.4607, copy_loss 0.01105
preds (tensor([1.1767, 1.1807, 1.1832], device='cuda:0'), tensor([1.1876, 1.1872, 1.1874], device='cuda:0'))
sharpe -7.5, ce_loss 0.6934, mse_loss 0.4014, copy_loss 0.01227
preds (tensor([1.1795, 1.1833, 1.1855], device='cuda:0'), tensor([1.1905, 1.1896, 1.1905], device='cuda:0'))
sharpe -1.1e+01, ce_loss 0.6938, mse_loss 0.1382, copy_loss 0.005751
preds (tensor([1.1765, 1.1805, 1.1830], device='cuda:0'), tensor([1.1973, 1.1966, 1.1970], device='cuda:0'))
sharpe -7.1, ce_loss 0.6935, mse_loss 0.5254, copy_loss 0.005302
preds (tensor([1.1744, 1.1785, 1.1813], device='cuda:0'), tensor([1.

Saving model checkpoint to ./results\checkpoint-1500
Configuration saved in ./results\checkpoint-1500\config.json
Model weights saved in ./results\checkpoint-1500\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1500, training_loss=9.067436218261719, metrics={'train_runtime': 1278.2607, 'train_samples_per_second': 2.347, 'train_steps_per_second': 1.173, 'total_flos': 0.0, 'train_loss': 9.067436218261719, 'epoch': 1.0})

In [10]:
del trainer
del model
torch.cuda.empty_cache()

# Appendix

## quick timing check

In [ ]:
model = GPT2Trader(config).cuda()

In [72]:
%%timeit
fake_data = torch.randn(4, 391, 256)
fake_data = fake_data.cuda()
model(fake_data)
cpu = fake_data.cpu()

28.2 ms ± 2.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [73]:
model = GPT2Trader(config).cpu()

Using 9 layers


In [74]:
%%timeit
fake_data = torch.randn(4, 391, 256)
model(fake_data)

748 ms ± 82.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
